# Project: Analyse @welovedogs tweets

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Datasets 
- An archive of @dog_rates tweets was provided by Udacity, it contains some information on the tweets eg tweet_id, text, name, ratings etc.
- A dataset containing the images included in the tweets, and thier image recognition result was also provided


> The following are questions to be explored using this dataset:
><li>Most popular dog name
><li>Dog breeds with the highest rating
><li>The Dog with the most retweets and likes
><li>Average overall rating 
><li>Average tweet engagement and at if its infleunced by time 

<a id='wrangling'></a>

## Data wrangling
<ul>
<li><a href="#gathering">Gathering Data</a></li>
<li><a href="#assess">Assessing Data</a></li>
<li><a href="#cleaning">Cleaning Data</a></li>
</ul>

### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import json
import tweepy
import re


<a id='gathering'></a>
### Gathering Data

In [2]:
tweets = pd.read_csv('twitter-archive-enhanced.csv')
image_predictions = pd.read_csv('image-predictions.tsv',sep = '\t')

Getting Twitter API's access_token and access_secret <br>
The consumer_key and secret are hidden to comply with Twitter's API terms and conditions

In [3]:
consumer_key = 'sVYfTGyNUkedyDsCdhWc54et9'
consumer_secret = 'khGtdnpBZV0WklSWfbFOOWbROhIiMYDso0WVaX5Z79WuXdXuWX'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)

access_token = auth.access_token
access_secret = auth.access_token_secret

auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

Obtaining the `tweet_ids` to use with the twitter api

In [4]:
id = tweets.tweet_id.values

Query Twitter API for each tweet in the Twitter archive and save JSON in a text file

In [337]:
fails = []
count = 1
with open('tweet-json.txt', 'w') as writefile:
    for x in id:
        print(str(count)+ ': ' + str(x))
        count += 1
        try:
            tweet = api.get_status(x, tweet_mode = 'extended')
            print('success')
            json.dump(tweet._json, writefile)
            writefile.write('\n')
        except tweepy.TweepError as error:
            print('fail')
            fails.append(str(x) + ': ' + str(error))
            pass
            

1: 892420643555336193


KeyboardInterrupt: 

Reading from the json text file created above 

In [5]:
tweet_info = pd.DataFrame()
i_d = []
favorite_count = []
retweet_count = []
followers = []
with open('tweet-json.txt', 'r') as json_data:
    for x in json_data:
        data = json.loads(x)
        i_d.append(data.get('id'))
        favorite_count.append(data.get('favorite_count'))
        retweet_count.append(data.get('retweet_count'))
        followers.append(data.get('user').get('followers_count'))
        print(data.get('user')['followers_count'])

8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8991106
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990741
8990742
8990742
8990742
8990742
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8991108
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990743
8990744
8990744
8990744
8990744
8990744
8990743
8990744
8990744
8990744
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990745
8990743
8990745
8990745
8990745
8990745
8990745


8990788
8990788
8990790
8990789
8990790
8990789
8990790
8990790
8990788
8990789
8990790
8990790
8990789
8990789
8990790
8990789
8990789
8990790
8990789
8990789
8990790
8990790
8990790
8990789
8990789
8990790
8990789
8990789
8990790
8990790
8990790
8990790
8990790
8990790
8990790
8990789
8990790
8990790
8990790
8990790
8990790
8990790
8990790
8990790
8990790
8990790
8990791
8990790
8990790
8990790
8990790
8990792
8990790
8990792
8990792
8990790
8990791
8990792
8990792
8990790
8990792
8990791
8990790
8990791
8990791
8990790
8990790
8990791
8990792
8990790
8990790
8990790
8990791
8990792
8990790
8990791
8990790
8990791
8990790
8990792
8990792
8990792
8990791
8990793
8990791
8990791
8990793
8990792
8990793
8990793
8990791
8990793
8990792
8990793
8990793
8990793
8990793
8990793
8990792
8990793
8990792
8990792
8990793
8990793
8990792
8990792
8990792
8990792
8990792
8990792
8990792
8990792
8990793
8990792
8990792
8990792
8990793
8990793
8990793
8990793
8990792
8990792
8990793
8990792
8990792


Writing the data into a dataframe called `tweet_info`

In [6]:
tweet_info['id'] = i_d
tweet_info['favorite_count'] = favorite_count
tweet_info['retweet_count'] = retweet_count
tweet_info['followers_count'] = followers

In [7]:
tweet_info

,id,favorite_count,retweet_count,followers_count
0,892420643555336193,34952,7334,8990741
1,892177421306343426,30266,5473,8990741
2,891815181378084864,22776,3620,8990741
3,891689557279858688,38227,7524,8990741
4,891327558926688256,36493,8095,8990741
...,...,...,...,...
2326,666049248165822465,93,38,8990852
2327,666044226329800704,259,122,8990852
2328,666033412701032449,106,39,8990852
2329,666029285002620928,117,41,8990852


<a id='assess'></a>

### Assessing Data


In [8]:
tweets.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1400,699446877801091073,NaN,NaN,2016-02-16 04:15:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He's undercover in all these p...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/699446877...,12,10,Archie,None,None,None,None
1451,695794761660297217,NaN,NaN,2016-02-06 02:22:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Wyatt. His throne is modeled after him...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/695794761...,13,10,Wyatt,None,None,None,None
38,884925521741709313,NaN,NaN,2017-07-12 00:01:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Earl. He found a hat. Nervous about wh...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/884925521...,12,10,Earl,None,None,None,None
414,822859134160621569,NaN,NaN,2017-01-21 17:31:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Hobbes. He was told he was going to th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/822859134...,12,10,Hobbes,None,None,None,None
1830,676219687039057920,NaN,NaN,2015-12-14 01:58:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kenneth. He's stuck in a bubble. 10/10...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676219687...,10,10,Kenneth,None,None,None,None


In [9]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [10]:
tweets.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [11]:
tweets.groupby('name').count()['tweet_id']

name
Abby            2
Ace             1
Acro            1
Adele           1
Aiden           1
               ..
such            1
the             8
this            1
unacceptable    1
very            5
Name: tweet_id, Length: 957, dtype: int64

In [12]:
image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [13]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [14]:
tweet_info.describe()

,id,favorite_count,retweet_count,followers_count
count,2.331000e+03,2331.000000,2331.000000,2.331000e+03
mean,7.419079e+17,7297.767482,2582.489918,8.990793e+06
std,6.823170e+16,11337.264701,4368.222352,5.050960e+01
min,6.660209e+17,0.000000,1.000000,8.990741e+06
25%,6.782670e+17,1262.000000,524.000000,8.990760e+06
50%,7.182469e+17,3161.000000,1205.000000,8.990789e+06
75%,7.986692e+17,8925.000000,2988.000000,8.990815e+06
max,8.924206e+17,150210.000000,74112.000000,8.991215e+06


In [15]:
tweet_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id               2331 non-null   int64
 1   favorite_count   2331 non-null   int64
 2   retweet_count    2331 non-null   int64
 3   followers_count  2331 non-null   int64
dtypes: int64(4)
memory usage: 73.0 KB


#### Quality
##### `tweets` table
- Missing dog names in the `name` column
- Retweeted tweets and replies in table
- `Timestamp` column is an object not a datetime 
- Presence of other animals other than dogs 
- wrong `name` (officially, by, the, infuriating, a, quite, not, one, my, actually, all, his, an, mad)
- Some rating denominator values are'nt 10

##### `image_predictions` table
- No dogs predictions in some of the rows

##### `tweet_info` table
- Retweeted tweets and replies in table
- Missing data

#### Tidiness
- One variables in four columns in `tweets` table (pupper, puppo, doggo, and floofer) are all dog_types
- Only one column needed for dog_breed prediction
- `tweet_info` and `tweets` need to be one column 

<a id='cleaning'></a>

### Cleaning Data

Creating new dataframes where the cleaning operations will be performed

In [533]:
tweets_clean = tweets.copy()
image_predictions_clean = image_predictions.copy()
tweet_info_clean = tweet_info.copy()

#### Tidiness
- One variables in four columns in `tweets` table (pupper, puppo, doggo, and floofer) are all dog_types<br>

**Define** <br>
Create a function to merge the four columns into 1 and delete them.

**Code**

In [534]:
def dog_type (a,b,c,d):
    if a == 'doggo':
        return (a)
    elif b == 'floofer':
        return (b)
    elif c == 'pupper':
        return (c)
    elif d == 'puppo':
        return (d)
    else:
        return(np.nan)

In [535]:
dog_types = map(dog_type,tweets_clean['doggo'],tweets_clean['floofer'],tweets_clean['pupper'],tweets_clean['puppo'])

In [536]:
tweets_clean['dog_types'] = list(dog_types)

In [537]:
tweets_clean = tweets_clean.drop(columns=['doggo','floofer','pupper','puppo'])

**Test**

In [538]:
tweets_clean.head(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_types
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,NaN
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,NaN
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,NaN
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,NaN
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,NaN
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo


#### Quality
- Retweeted tweets and replies in table<br>

**Define** <br>
Remove the retweets and replies from the dataframe.

**Code**

In [539]:
tweets_clean = tweets_clean.query('in_reply_to_status_id != in_reply_to_status_id and retweeted_status_id != retweeted_status_id')

In [540]:
tweets_clean.drop(columns=['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], inplace= True)

**Test**

In [541]:
tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2097 non-null   int64 
 1   timestamp           2097 non-null   object
 2   source              2097 non-null   object
 3   text                2097 non-null   object
 4   expanded_urls       2094 non-null   object
 5   rating_numerator    2097 non-null   int64 
 6   rating_denominator  2097 non-null   int64 
 7   name                2097 non-null   object
 8   dog_types           336 non-null    object
dtypes: int64(3), object(6)
memory usage: 163.8+ KB


#### Quality
- Timestamp column is an object not a datetime<br>

**Define** <br>
Change timestamp datatype to datetime using pandas `to_datetime` function.

**Code**

In [542]:
tweets_clean['timestamp'] = pd.to_datetime(tweets_clean.timestamp)

**Test**

In [544]:
tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2097 non-null   int64              
 1   timestamp           2097 non-null   datetime64[ns, UTC]
 2   source              2097 non-null   object             
 3   text                2097 non-null   object             
 4   expanded_urls       2094 non-null   object             
 5   rating_numerator    2097 non-null   int64              
 6   rating_denominator  2097 non-null   int64              
 7   name                2097 non-null   object             
 8   dog_types           336 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 163.8+ KB


#### Quality
- Missing dog names in the name column
- Wrong name (officially, by, the, infuriating, a, quite, not, one, my, actually, all, his, an, mad)<br>

**Define** <br>
Create function to get all dog names after string `name is` and `named` in the text column.<br>
Prevent words such as `a`,`an`,`by`,`not`,`my` etc from being in the name column.<br>
All words('a','an','by','not','my','life','old','just','very', 'this','light','getting','unacceptable','space','None','officially','the','infuriating','such','quite', 'one', 'actually', 'all', 'his','incredibly','mad')


**Code**

In [545]:
def get (data):
    tweet_id = []
    indexes =[]
    names =[]
    for index,df in data.iterrows():
        name = re.findall(r'(?<=\bThis is\s)([A-Z]\w+)|(?<=\bname is\s)([A-Z]\w+)|(?<=\bnamed\s)([A-Z]\w+)|(?<=\bMeet\s)([A-Z]\w+)|(?<=\bHere is\s)([A-Z]\w+)|(?<=\bhello to\s)([A-Z]\w+)|(?<=\bthis is\s)([A-Z]\w+)',df['text'])
        for x in name:
            indexes.append(index)
            tweet_id.append(df['tweet_id'])
            strin = re.findall(r'(\w+)', str(x))
            names.append(strin[0])
    return tweet_id,indexes,names

In [546]:
tw_id, index, names = get(tweets_clean)

In [547]:
new_name = pd.DataFrame(index = index, data={'name' : names,'tweet_id':tw_id})
new_name

,name,tweet_id
0,Phineas,892420643555336193
1,Tilly,892177421306343426
2,Archie,891815181378084864
3,Darla,891689557279858688
4,Franklin,891327558926688256
...,...,...
2315,Christoper,666691418707132416
2317,Jimothy,666644823164719104
2318,Kreggory,666454714377183233
2319,Scout,666447344410484738


In [529]:
tweets_clean = tweets_clean.merge(new_name,left_on ='tweet_id',right_on='tweet_id',how = 'right')

In [531]:
tweets_clean.head(100)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name_x,dog_types,name_y
0,892420643555336193,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,Phineas
1,892177421306343426,2017-08-01 00:17:27+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN,Tilly
2,891815181378084864,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN,Archie
3,891689557279858688,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,NaN,Darla
4,891327558926688256,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,NaN,Franklin
...,...,...,...,...,...,...,...,...,...,...
95,863553081350529029,2017-05-14 00:34:33+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Neptune. He's a backpup vocalist for t...,https://twitter.com/dog_rates/status/863553081...,13,10,Neptune,NaN,Neptune
96,863432100342583297,2017-05-13 16:33:49+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Belle. She's never been more pupset. E...,https://twitter.com/dog_rates/status/863432100...,12,10,Belle,NaN,Belle
97,863062471531167744,2017-05-12 16:05:02+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Quinn. She's quite the goofball. ...,"https://www.gofundme.com/helpquinny,https://tw...",13,10,Quinn,NaN,Quinn
98,862831371563274240,2017-05-12 00:46:44+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Zooey. She's the world's biggest fan o...,https://twitter.com/dog_rates/status/862831371...,13,10,Zooey,NaN,Zooey
